In [36]:
'''binary encoding'''
def bin_enc(lbl):
  mi = min(lbl)
  length = len(bin(max(lbl)-mi+1)[2:])
  enc=[]

  for i in lbl:
    b=bin(i-mi)[2:].zfill(length)
    enc.append([int(n) for n in b])

  return enc  

def bin_dec(enc, mi=0):
  lbl=[]

  for e in enc:
    rounded=[int(round(x)) for x in e]
    string= ''.join(str(x) for x in rounded)
    num=int(string,2) + mi
    lbl.append(num)
    
  return lbl


In [37]:
#one-hot encoding

import numpy as np
def onehot_enc(lbl, min_val=0):
  mi=min(lbl)
  enc=np.full((len(lbl),max(lbl)-mi+1), min_val, np.int8)
  for i, x in enumerate(lbl):
    enc[i, x-mi]=1
  return enc

def onehot_dec(enc, mi=0):
  return [np.argmax(e)+mi for e in enc]

In [272]:
labels= 1, 2, 3, 4, 5
enc = onehot_enc(labels)
dec = onehot_dec(enc, min(labels))

print(enc)
print(dec)

[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]
[1, 2, 3, 4, 5]


In [39]:
'''Fungsi aktivasi sigmoid dan turunannya'''

def sig(X):
 return [1/(1+np.exp(-x)) for x in X]

def sigd(X):
 output=[]
 for i, x in enumerate(X):
   s = sig([x])[0]
   output.append(s*(1-s))

 return output


In [40]:
'''fungsi modeling (training backpropagation)'''
def bp_fit(X, target, layer_conf, max_epoch, max_error=.1, learn_rate=.1,print_per_epoch=100):
  nin=[np.empty(i) for i in layer_conf]

  n = [np.empty(j+1) if i<len(layer_conf)-1
      else np.empty(j) for i, j in enumerate(layer_conf)]
      
  w = np.array([np.random.rand(layer_conf[i]+1, layer_conf[i+1])
                for i in range(len(layer_conf)-1)])
  
  dw = [np.empty((layer_conf[i]+1, layer_conf[i+1]))
        for i in range(len(layer_conf)-1)]
        
  d = [np.empty(s) for s in layer_conf[1:]]
  din = [np.empty(s) for s in layer_conf[1:-1]]
  epoch = 0
  mse = 1

  for i in range(0, len(n)-1):
    n[i][-1]=1
  while (max_epoch == -1 or epoch<max_epoch) and mse>max_error:
    epoch +=1
    mse = 0
    for r in range(len(X)):
      n[0][:-1]=X[r]

      for L in range(1, len(layer_conf)):
        nin[L] = np.dot(n[L-1], w[L-1])
        n[L][:len(nin[L])]=sig(nin[L])

      e = target[r] - n[-1]
      mse += sum(e ** 2)
      d[-1]=e*sigd(nin[-1])
      dw[-1]=learn_rate * d[-1]*n[-2].reshape((-1,1))

      for L in range(len(layer_conf)-1, 1, -1):
        din[L-2]=np.dot(d[L-1], np.transpose(w[L-1][:-1]))
        d[L-2]=din[L-2]*np.array(sigd(nin[L-1]))
        dw[L-2]=(learn_rate*d[L-2])*n[L-2].reshape((-1,1))

      w += dw
    mse /= len(X)

  if print_per_epoch > -1 and epoch % print_per_epoch == 0:
    print(f'Epoch {epoch}, MSE: {mse}')

  return w, epoch, mse


In [41]:
'''fungsi pengujian back propagation'''

def bp_predict(X,w):
  n=[np.empty(len(i)) for i in w]
  nin=[np.empty(len(i[0])) for i in w]
  predict = []
  n.append(np.empty(len(w[-1][0])))

  for x in X:
    n[0][:-1]=x

    for L in range(0, len(w)):
      nin[L] = np.dot(n[L], w[L])
      n[L+1][:len(nin[L])] = sig(nin[L])

    predict.append(n[-1].copy())
  
  return predict


In [271]:
# Import library
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

# Load data, pisahkan data dan label

#iris = datasets.load_iris()
#X = minmax_scale(iris.data)
#Y = onehot_enc(iris.target)
seeds_dataset = np.loadtxt('seeds_dataset.txt')

data = seeds_dataset[:, :7]
labels = seeds_dataset[:, 7].reshape((data.shape[0]))

label = []
for i in range (len(labels)) :
  label.append(int(labels[i]))

X = minmax_scale(data)
Y = onehot_enc(label)

# Preprocess data: membagi data training, label training dan data testing, label testing (2/3 data training, 1/3 data testing)
# Atau dengan kata lain Memisahkan data training, data testing, label training, label testing (2/3 data training, 1/3 data testing)

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=.3)

# Membangun model JST / backpropagation menggunakan data training dan label training

w, ep, mse = bp_fit(X_train, y_train, layer_conf=(7,10,3),learn_rate=.1, max_epoch=1000, max_error=.1, print_per_epoch=25)

print(f'Epochs: {ep}, MSE: {mse}')

# Menguji model / Menguji kinerja backpropagation menggunakan data testing
predict = bp_predict(X_test, w)
predict = onehot_dec(predict)

predict1 = []
for i in range (len(predict)):
  predict1.append(predict[i]+1)

#Membandingkan luaran dengan label testing / target
y_test = onehot_dec(y_test)
acc = accuracy_score(predict, y_test)

y_test1 = []
for i in range (len(y_test)):
  y_test1.append(y_test[i]+1)

print(len(X_test))
print(len(X_train))
print(f'Output: {predict1}')
print(f'True: {y_test1}')
print(f'Accuracy: {acc}')

Epochs: 208, MSE: 0.09979312296157221
63
147
Output: [2, 1, 3, 3, 3, 1, 1, 3, 1, 2, 3, 1, 2, 3, 3, 2, 2, 1, 2, 3, 1, 2, 1, 3, 3, 3, 3, 1, 3, 3, 3, 3, 1, 1, 3, 1, 3, 1, 3, 1, 3, 1, 2, 3, 2, 1, 2, 1, 2, 3, 1, 1, 2, 1, 2, 3, 3, 1, 3, 1, 1, 3, 2]
True: [2, 1, 3, 3, 3, 1, 1, 3, 1, 2, 3, 1, 2, 3, 3, 2, 2, 1, 2, 3, 1, 2, 1, 3, 3, 3, 3, 1, 3, 3, 3, 3, 1, 1, 3, 1, 3, 1, 3, 1, 3, 1, 2, 3, 2, 1, 2, 1, 2, 3, 2, 1, 2, 1, 2, 3, 3, 1, 3, 1, 1, 3, 2]
Accuracy: 0.9841269841269841


In [261]:
'''Menguji label'''

from sklearn import datasets
seeds_dataset = np.loadtxt('seeds_dataset.txt')

dataz = seeds_dataset[:, :7]
labels = seeds_dataset[:, 7]

label = []
for i in range (len(labels)) :
  label.append(int(labels[i]))

#print(dataz)
print(label)

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(label)

Y1 = onehot_dec(Y)
Y2 = []

#print(onehot_enc(label))
for i in range (len(Y1)):
  Y2.append(Y1[i]+1)

print(Y2)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [222]:
'''Jika ingin mengelompokkan data test dan data trainibng sendiri'''

import numpy as np
from sklearn.model_selection import KFold
X = np.array(data)
y = np.array(label)
kf = KFold(n_splits=3,shuffle=True)


for train_index, test_index in kf.split(X):
  #print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

print(len(X_train))
print(len(X_test))


140
70
